# 한국어-영어 번역(병렬) 말뭉치 AI 데이터

구어체 데이터 40만 개를 사용하여 Transformer 번역 모델을 학습합니다.


링크: https://aihub.or.kr/aidata/87

In [1]:
# %load_ext lab_black

In [2]:
import pandas as pd

In [3]:
path_1 = "./data/1_구어체(1)_200226.xlsx"
path_2 = "./data/1_구어체(2)_200226.xlsx"

In [4]:
data_1 = pd.read_excel(path_1, engine="openpyxl")
data_2 = pd.read_excel(path_2, engine="openpyxl")

/home/youngerous/.local/lib/python3.6/site-packages/openpyxl/styles/stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [5]:
print(len(data_1), len(data_2))

200000 200000


In [6]:
data_1.head()

,SID,원문,번역문
0,1,'Bible Coloring'은 성경의 아름다운 이야기를 체험 할 수 있는 컬러링 ...,Bible Coloring' is a coloring application that...
1,2,씨티은행에서 일하세요?,Do you work at a City bank?
2,3,푸리토의 베스트셀러는 해외에서 입소문만으로 4차 완판을 기록하였다.,"PURITO's bestseller, which recorded 4th rough ..."
3,4,11장에서는 예수님이 이번엔 나사로를 무덤에서 불러내어 죽은 자 가운데서 살리셨습니다.,In Chapter 11 Jesus called Lazarus from the to...
4,5,"6.5, 7, 8 사이즈가 몇 개나 더 재입고 될지 제게 알려주시면 감사하겠습니다.",I would feel grateful to know how many stocks ...


In [11]:
del data_2["Unnamed: 3"]
data_2.head()

,SID,원문,번역문
0,200001,0 설정을 입력하고 안정될 때까지 5분 동안 기다린 후 OK 버튼을 길게 누르십시오.,"Enter into 0 setting, and wait for 5 minutes t..."
1,200002,0은 그들에게 아무것도 아니었지만 무는 숫자일 수가 없습니다.,The zero was nothing for them but nothing coul...
2,200003,"1,015버전에서 핫키 버그가 있습니다.","There is a Hotkey bug in the 1,015 version."
3,200004,"1,390점에서 1,440점을 득점한 사람은 재판을 위해 걸러집니다.","Individuals who got a score between 1,390 and ..."
4,200005,"1,400년보다 오래 전의 유적지에 있는 최초의 성당에서 숭배자들은 그것을 인지했을...","Indeed, worshippers at the very first cathedra..."


# Tokenizer
huggingface에서 제공하는 multilingual 토크나이저(```bert-base-multilingual-cased```)를 사용합니다.

In [12]:
from transformers import BertTokenizer

In [13]:
tok = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

In [14]:
vocab_list = list(tok.vocab.items())

In [15]:
vocab_list[:150]

[('[PAD]', 0),
 ('[unused1]', 1),
 ('[unused2]', 2),
 ('[unused3]', 3),
 ('[unused4]', 4),
 ('[unused5]', 5),
 ('[unused6]', 6),
 ('[unused7]', 7),
 ('[unused8]', 8),
 ('[unused9]', 9),
 ('[unused10]', 10),
 ('[unused11]', 11),
 ('[unused12]', 12),
 ('[unused13]', 13),
 ('[unused14]', 14),
 ('[unused15]', 15),
 ('[unused16]', 16),
 ('[unused17]', 17),
 ('[unused18]', 18),
 ('[unused19]', 19),
 ('[unused20]', 20),
 ('[unused21]', 21),
 ('[unused22]', 22),
 ('[unused23]', 23),
 ('[unused24]', 24),
 ('[unused25]', 25),
 ('[unused26]', 26),
 ('[unused27]', 27),
 ('[unused28]', 28),
 ('[unused29]', 29),
 ('[unused30]', 30),
 ('[unused31]', 31),
 ('[unused32]', 32),
 ('[unused33]', 33),
 ('[unused34]', 34),
 ('[unused35]', 35),
 ('[unused36]', 36),
 ('[unused37]', 37),
 ('[unused38]', 38),
 ('[unused39]', 39),
 ('[unused40]', 40),
 ('[unused41]', 41),
 ('[unused42]', 42),
 ('[unused43]', 43),
 ('[unused44]', 44),
 ('[unused45]', 45),
 ('[unused46]', 46),
 ('[unused47]', 47),
 ('[unused48]', 

In [16]:
tok.tokenize("안녕하세요")

['안', '##녕', '##하', '##세', '##요']

# Train, Valid, Test Split

In [23]:
from sklearn.model_selection import train_test_split

In [17]:
# concatenate two dataframe
total_data = pd.concat([data_1, data_2], axis=0).reset_index(drop=True)
len(total_data)

400000

In [19]:
del total_data["SID"]
total_data.tail()

,원문,번역문
399995,당신은 곧 당신의 조카를 볼 수 있어요.,You can see your nephew soon.
399996,나는 당신이 내게 공유한 음악을 들었어요.,I've listened to the music that you shared wit...
399997,그 서류는 당신이 공유해 준 것과 달라요.,The document is different from what you shared.
399998,당신이 잡지에 광고 낸 것을 봤어요.,I saw your advertisement in the magazine.
399999,당신이 이제 괜찮다니 저는 참 기쁘군요.,I'm really glad you're all right now.


In [26]:
# split
trn_val, test = train_test_split(total_data, test_size=0.15, shuffle=True)
print(len(trn_val), len(test))

340000 60000


In [27]:
train, valid = train_test_split(trn_val, test_size=0.15, shuffle=True)
print(len(train), len(valid))

289000 51000


Train 289,000 / Valid 51,000 / Test 60,000

In [28]:
# save file
train.to_csv("./data/train_289000.csv", index=False)
valid.to_csv("./data/valid_51000.csv", index=False)
test.to_csv("./data/test_60000.csv", index=False)

### check saved file

In [29]:
df_trn = pd.read_csv("./data/train_289000.csv")

In [30]:
len(df_trn)

289000

In [31]:
df_trn.head()

,원문,번역문
0,친구가 함께한다면 나는 힘들 때 위로받을 수 있어요.,I can be cheered up if I am with my friends.
1,나는 이번 주에 예비군 훈련하러 가요.,I'm going to reserved army drill this week.
2,독도는 지리적으로 일본보다 한국에 더 가깝습니다.,Dokdo is geographically closer to Korea than J...
3,하기 그림부분 SM이 겹쳐져 있습니다.,The SM of the picture part is overlapped in th...
4,우리는 3명이 남아 있습니다.,We have 3 people left.


In [32]:
tok.tokenize("친구가 함께한다면 나는 힘들 때 위로받을 수 있어요.")

['친',
 '##구',
 '##가',
 '함께',
 '##한다',
 '##면',
 '나는',
 '힘',
 '##들',
 '때',
 '위',
 '##로',
 '##받',
 '##을',
 '수',
 '있어',
 '##요',
 '.']

In [33]:
tok.tokenize("I can be cheered up if I am with my friends.")

['I',
 'can',
 'be',
 'che',
 '##ered',
 'up',
 'if',
 'I',
 'am',
 'with',
 'my',
 'friends',
 '.']